# An Application of Deep Reinforcement Learning to Algorithmic Trading

By Thibaut Théatea, Damien Ernsta

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
%run datadownloader.py

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
training_data = pd.read_csv('data/AAPL_2012-01-01_2017-12-31.csv')
training_data

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2012-01-03,14.686785697937012,14.73214340209961,14.60714340209961,14.621429443359375,302220800
3,2012-01-04,14.765713691711426,14.8100004196167,14.617142677307129,14.64285659790039,260022000
4,2012-01-05,14.929642677307129,14.948213577270508,14.738213539123535,14.819643020629883,271269600
...,...,...,...,...,...,...
1506,2017-12-22,43.752498626708984,43.85499954223633,43.625,43.66999816894531,65397600
1507,2017-12-26,42.64250183105469,42.86750030517578,42.41999816894531,42.70000076293945,132742000
1508,2017-12-27,42.650001525878906,42.69499969482422,42.4275016784668,42.525001525878906,85992800
1509,2017-12-28,42.77000045776367,42.962501525878906,42.619998931884766,42.75,65920800


In [5]:
import yfinance as yf

In [25]:
tickers = ["AAPL"]
start_date = "2012-01-01"
start_validation_date = "2016-01-01"

aapl = yf.download("AAPL", start=start_date, end=start_validation_date)
aapl

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2012-01-03,12.375388,14.686786,14.732143,14.607143,14.621429,302220800
2012-01-04,12.441897,14.765714,14.810000,14.617143,14.642857,260022000
2012-01-05,12.580029,14.929643,14.948214,14.738214,14.819643,271269600
2012-01-06,12.711535,15.085714,15.098214,14.972143,14.991786,318292800
2012-01-09,12.691375,15.061786,15.276786,15.048214,15.196429,394024400
...,...,...,...,...,...,...
2015-12-24,24.440699,27.007500,27.250000,26.987499,27.250000,54281600
2015-12-28,24.166946,26.705000,26.922501,26.545000,26.897499,106816800


In [26]:
aapl.columns = aapl.columns.droplevel(1)

In [27]:
aapl

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2012-01-03,12.375388,14.686786,14.732143,14.607143,14.621429,302220800
2012-01-04,12.441897,14.765714,14.810000,14.617143,14.642857,260022000
2012-01-05,12.580029,14.929643,14.948214,14.738214,14.819643,271269600
2012-01-06,12.711535,15.085714,15.098214,14.972143,14.991786,318292800
2012-01-09,12.691375,15.061786,15.276786,15.048214,15.196429,394024400
...,...,...,...,...,...,...
2015-12-24,24.440699,27.007500,27.250000,26.987499,27.250000,54281600
2015-12-28,24.166946,26.705000,26.922501,26.545000,26.897499,106816800
2015-12-29,24.601336,27.184999,27.357500,26.715000,26.740000,123724800
